# Produces dataset of project times from appended pipeline datasets ("all_quarters_merged.csv")

In [1]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

In [2]:
logging.basicConfig(level=logging.INFO)

# Actual Code

In [5]:
def convert_to_one_record_per_project(df):
    """
    Group the dataset by (apn, address) and then emit one row per (apn, address) pair.
    best_date and best_stat will be converted into arrays.
    All other attributes will be taken from the record with the most recent `best_date` attribute.
    """
    gb = df.groupby(['apn', 'address'])
    for k in gb.groups:
        group_df = gb.get_group(k)
        group_df = group_df.sort_values(['best_date', 'report_year', 'report_quarter'], ascending=[True, True, True])
        last_row = group_df.tail(1).copy()
        
        # identify building permit ID. Then fill in rest of quarters with this permit ID.
        building_permit=np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
                building_permit=building_permit
            else:
                building_permit = row['dbi_permit']
                
        #Identify completion quarter for those projects that have reached completion
        for index, row in group_df.iterrows():
            if row['best_stat'] !='CONSTRUCTION':
                comp_quarter= np.nan
                comp_year=np.nan
            elif row['best_stat'] == 'CONSTRUCTION':
                comp_quarter = row['report_quarter']
                comp_year = row['report_year']

        if pd.isnull(comp_quarter):
            pass
        if comp_quarter == 1 and comp_year == 2017:
            comp_quarter = np.nan
        elif comp_quarter == 4:
            comp_quarter = 1
            comp_year = 1+comp_year
        else:
            comp_quarter = 1+comp_quarter
            
        if comp_quarter ==1:
            comp_daymth= '01/01'
        elif comp_quarter ==2:
            comp_daymth= '04/01'
        elif comp_quarter==3:
            comp_daymth= '07/01'
        elif comp_quarter == 4:
            comp_daymth= '10/01'
        elif pd.isnull(comp_quarter):
            comp_daymth=np.nan

        if pd.isnull(comp_daymth):
            comp_date = np.nan
        else:
            comp_date = comp_daymth + "/" + str(comp_year)
        
        #Identify earliest "firstfiled" date
        firstfiled=''
        for index, row in group_df.iterrows():
            if pd.isnull(row['firstfiled']):
                continue
            else:
                if len(firstfiled) ==0:
                    firstfiled=row['firstfiled']
                else:
                    if dateutil.parser.parse(row['firstfiled']) < dateutil.parser.parse(firstfiled):
                        firstfiled=row['firstfiled']
                    else: 
                        firstfiled=firstfiled
        
        #Identify earliest best date
        earliest_BD = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['best_date']):
                continue
            else:
                if len(earliest_BD) == 0:
                    earliest_BD = row['best_date']
                else:
                    continue
        
        # Finalize first date variable (minimum of earliest best_date and firstfiled)
        if firstfiled =='' and earliest_BD !='':
            first_date = earliest_BD
        elif earliest_BD=='' and firstfiled !='':
            first_date = firstfiled
        elif firstfiled !='' and earliest_BD !='':
            first_date = min(firstfiled, earliest_BD)
            
        # initiate variables. Groups without these dates are blank for these variables.
        BP_date = ''
        con_date = ''

        #Identify first date for all status categories
        m=0
        for index, row in group_df.iterrows():
            if m == 0:
                status_previous = 'blah'
            if row['status']=='Building Permit Approved':
                if index == 0:
                    BP_date = row['best_date']
                elif index !=0:
                    if status_previous =='Building Permit Approved':
                        BP_date=BP_date
                    else:
                        BP_date = row['best_date']
            elif row['status']=='Under Construction':
                if index == 0:
                    con_date = row['best_date']
                elif index !=0:
                    if status_previous =='Under Construction':
                        con_date=con_date
                    else:
                        con_date = row['best_date']
            status_previous = row['status']
            m=m+1
            
        #Identify latest unit counts
        units = np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['units']):
                continue
            else:
                units = row['units']
                
        #Identify latest net unit counts
        unitsnet= np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['unitsnet']):
                continue
            else:
                unitsnet = row['unitsnet']
        
        #a few projects have construction best date after the completion date. In these cases, match the two.
        if pd.notnull(comp_date) & pd.notnull(con_date):
            if dateutil.parser.parse(comp_date) < dateutil.parser.parse(con_date):
                comp_date = con_date
                
        #Identify latest zoning designation
        for index, row in group_df.iterrows():
            zoning = row['zoning_simplified']
        
        last_row['firstfiled']=firstfiled
        last_row['dbi_permit']= building_permit
        last_row['comp_date']=comp_date
        last_row['BP_date'] = BP_date
        last_row['con_date'] = con_date
        last_row['first_date']=first_date
        last_row['latest_project_record_date'] = last_row.best_date
        last_row['first_project_record_date'] = group_df.iloc[0].best_date
        last_row['latest_project_status'] = last_row.best_stat
        last_row['units']=units
        last_row['unitsnet']=unitsnet
        last_row['zoning']=zoning

        ## Store a parseable list of all the project states and the dates those states were reported
        last_row['project_dates'] = str(tuple(group_df.best_date))
        last_row['project_statuses'] = str(tuple(group_df.best_stat))

        ## Store the project duration in days
        if not (pd.isnull(comp_date) or pd.isnull(first_date)):
            last_row['project_duration_days'] = (dateutil.parser.parse(comp_date) - dateutil.parser.parse(first_date)).days
        
        yield last_row


def main():
    df = pd.read_csv("cleaned/all_quarters_merged.csv")
    #consolidate status categories. Start with 3 for now
    def status_function(value):
        if value['best_stat']=="CONSTRUCTION":
            field = 'Under Construction'
        elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
            field = 'Building Permit Approved'
        else:
            field = 'Proposed'
        return field

    df['status']=df.apply(status_function, axis=1)
    
    #keep only variables we want
    list = ['address', 'apn', 'best_date', 'best_stat','status', 'firstfiled', 'report_quarter', 'report_year','units','unitsnet', 'dbi_permit', 'zoning_simplified', 'x', 'y']
    df = df[list]
    new_df = pd.concat(convert_to_one_record_per_project(df))
    logging.info("Writing output ({} rows, {} cols) to data/cleaned/all_quarters__one_record_per_project.csv".format(*new_df.shape))
    new_df.to_csv("cleaned/all_quarters__one_record_per_project.csv")
    return new_df

In [6]:
new_df = main()

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (1,2,6,8,12,13,16,17,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,46,48,49,52,53,54,56,61,63,64,65,66,67,68,69,70,72,76,79,80,81,82,83,85,86,87,88,89,90,91,92,93,94,95,100,102,104,105,106,107,108,110,111,112,113,114,117,119,126,129,130,131,132,134,135) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [126]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
new_df = new_df[new_df['units'] > 0]

In [127]:
#Next, keep only those that reached completion at some point over the time period
new_df=new_df[pd.notnull(new_df['comp_date'])]
new_df=new_df[new_df['firstfiled'] !='']

In [128]:
new_df=new_df[pd.notnull(new_df['firstfiled'])]

# Manual Data Fixes (Post Catching Mistake)

In [134]:
# fix duplicates
new_df['address_apn']=new_df['address']+new_df['apn']

for index, row in new_df.iterrows():
    if row['dbi_permit']=='':
        row['dbi_permit']==row['address_apn']

In [146]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address_apn'].unique()

array(['2101 & 2155 WEBSTER STREET0629037', '70 DOUGLASS ST2625032',
       '3575 GEARY BL1083002', '1006 16TH ST3833002',
       '800 BROTHERHOOD WAY (UNDER CONSTRUCTION)7331005',
       'HUDSON AVE AND WHITNEY YOUNG CIR4711118',
       '1100 GOLDEN GATE AV0757025', '55 LAGUNA STREET0857001',
       '1201-1225 TENNESSEE STREET4172022', '443 LINDEN ST0818049',
       '49 MOSS ST3731119', '1169 MARKET ST3702053',
       '782-786 ANDOVER ST5825007', '746 CAROLINA ST4096005',
       '800 BROTHERHOOD WAY7331005', '1301 INDIANA ST4228010',
       '145 AMES ST3632038', '3150 GEARY BL1066060', '4 OCTAVIA ST0855011',
       '338 CARL ST1265009', '1076 HAMPSHIRE ST4152016',
       '690 MONTEREY BLVD3094055', '1000 16TH ST3833001',
       '236 - 238 SHIPLEY ST / 77 FALMOUTH ST3753314',
       '1078 HAMPSHIRE ST4152046', '47 MOSS ST3731119',
       '55 LAGUNA ST (BLDG 1)0870003', '1400 MISSION ST3507039',
       '1006 / 1050 16TH ST & 1380 07TH ST3833002', '3575 GEARY BL1084010',
       '1068 GUE

In [147]:
new_df[new_df['address_apn'] == '3575 GEARY BL1083002']['dbi_permit']

1058    200702224724
Name: dbi_permit, dtype: object

In [148]:
new_df[new_df['dbi_permit']=='200702224724']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,address_apn
1058,,3575 GEARY BL,1083002,2009-12-14,CONSTRUCTION,01/01/2010,2009-08-19,200702224724,2003-05-20,2009-08-19,2003-05-20,2009-12-14,CONSTRUCTION,"('2009-08-19', '2009-12-14')",2418.00,"('CONSTRUCTION', 'CONSTRUCTION')",4,2009,Under Construction,150.00,150.00,-122.46,37.78,3575 GEARY BL1083002
5422,,3575 GEARY BL,1084010,2010-09-24,CONSTRUCTION,04/01/2011,2010-04-26,200702224724,2003-05-20,2010-04-26,2003-05-20,2010-09-24,CONSTRUCTION,"('2010-04-26', '2010-05-07', '2010-08-16', '20...",2873.00,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",1,2011,Under Construction,150.00,nan,-122.46,37.78,3575 GEARY BL1084010


# Previous Manual Fixes (Pre-Mistake)

In [116]:
# Pre Mistake Manual Fixes

# fix duplicates
new_df['address_apn']=new_df['address']+new_df['apn']

#decided that both this and its duplicate BP at 1169 Market Street are two phases of the same project. Revise start date and drop 1169 Market
new_df.loc[(new_df['address']=='1190 MISSION ST') & (new_df['apn']=='3702052'), 'first_date'] = '2003-07-15'
new_df = new_df[new_df['address_apn'] !='1169 MARKET ST3702053']

#Same project. Same dates except for construction date. Choosing best of the two dates based on Google street view.
new_df.loc[(new_df['address']=='49 MOSS ST') & (new_df['apn']=='3731119'), 'con_date'] = '2015-05-20'
new_df = new_df[new_df['address_apn'] !='47 MOSS ST3731119']

#Assume that 101 EXECUTIVE PARK BL duplicates are legitimately separate projects. Different unit counts, etc.

#Changed address name in final Q4 2015. Extend completion date by 1 quarter. Drop the duplicate.
new_df.loc[(new_df['address']=='690 MONTEREY BLVD') & (new_df['apn']=='3094055'), 'comp_date'] = '2015-05-20'
new_df=new_df[new_df['address_apn'] !='690 MONTEREY BL3094055']

#1340-1390 MISSION ST no dbi permit in any quarter just leave in

#Multiple Hunters point shipyards. I extend completion date and change units to latest available information. 
new_df.loc[(new_df['address']=='HUNTERS POINT SHIPYARD PHASE I') & (new_df['apn']=='4591C001'), 'comp_date'] = '10/01/2016'
new_df.loc[(new_df['address']=='HUNTERS POINT SHIPYARD PHASE I') & (new_df['apn']=='4591C001'), 'units'] = 167
new_df=new_df[new_df['address_apn'] !='HP SHIPYARD PHASE 1 (UNDER CONSTRUCTION)4591C001']

#Same project. Change construction date to reflect earliest information.
new_df.loc[(new_df['address']=='800 BROTHERHOOD WAY (UNDER CONSTRUCTION)') & (new_df['apn']=='7331005'), 'con_date'] = '2012-09-11'
new_df=new_df[new_df['address_apn'] !='800 BROTHERHOOD7331005']

#HUDSON AVE AND WHITNEY YOUNG CIR4711118—no dbi permit information in any quarter. Just leave in

#Same project. Change completion date to latest available info.
new_df.loc[(new_df['address']=='MARKET OCTAVIA - PARCEL P') & (new_df['apn']=='0831023'), 'comp_date'] = '01/01/2016'
new_df=new_df[new_df['address_apn'] !='PARCEL P - MARKET OCTAVIA0831023']

#duplicate projects. Update to reflect latest completion date information.
new_df=new_df[new_df['address_apn'] !='443 LINDEN ST0818049']

#duplicate projects. Update completion date and units to most recent information. Everything else to earliest info.
new_df.loc[(new_df['address']=='2101 & 2155 WEBSTER STREET') & (new_df['apn']=='0629037'), 'con_date'] = '2015-03-18'
new_df=new_df[new_df['address_apn'] !='2155 WEBSTER ST0629037']

#duplicate projects but one just pops up for one quarter so just drop it.
new_df=new_df[new_df['address_apn'] !='70 DOUGLASS ST2625032']

#duplicate projects. Update completion date to most recent. Update construction date to earliest.
new_df.loc[(new_df['address']=='3575 GEARY BL') & (new_df['apn']=='1084010'), 'con_date'] = '2010-04-26'
new_df=new_df[new_df['address_apn'] !='3575 GEARY BL1083002']

#duplicate projects. Drop ones that are out of date.
new_df=new_df[new_df['address_apn'] !='1100 GOLDEN GATE AV0757028']
new_df=new_df[new_df['address_apn'] !='1239 TURK ST0757027']

#duplicate projects. Update completion date to most recent. Update construction date to earliest available.
#Determined through PIM that 218 Buchanan alteration to 55 Laguna Street project. Update information to earliest possible.
new_df.loc[(new_df['address']=='55 LAGUNA ST') & (new_df['apn']=='0857001'), 'con_date'] = '2013-08-07'
new_df.loc[(new_df['address']=='55 LAGUNA ST') & (new_df['apn']=='0857001'), 'BP_date'] = '2013-06-11'
new_df=new_df[new_df['address_apn'] !='55 LAGUNA STREET0857001']
new_df=new_df[new_df['address_apn'] !='218 BUCHANAN ST0857001A']

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='782-786 ANDOVER ST5825032']

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='746 CAROLINA ST4096109']

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='1301 INDIANA ST4228010']

#145 AMES ST and 1068 GUERRERO ST seem like legitimately different projects despite duplicate building permits.

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='3150 GEARY BL1066060']

#seems like the same project (8 Octavia versus 4 Octavia). Updating completion date.
new_df.loc[(new_df['address']=='4 OCTAVIA ST') & (new_df['apn']=='0855011'), 'BP_date'] = '10/01/2014'
new_df=new_df[new_df['address_apn'] !='8 OCTAVIA ST0855011']

#drop duplicate 338 Carl Street. No new info.
new_df=new_df[new_df['address_apn'] !='338 CARL ST1265047']

#drop duplicate 1076 Hampshire Street and 1078 Hampshire Street. No new info.
new_df=new_df[new_df['address_apn'] !='1076 HAMPSHIRE ST4152045']
new_df=new_df[new_df['address_apn'] !='1078 HAMPSHIRE ST4152046']

#duplicates. Updating comp date to most recent info.
new_df.loc[(new_df['address']=='1420 MISSION ST') & (new_df['apn']=='3507039'), 'comp_date'] = '04/01/2015'
new_df=new_df[new_df['address_apn'] !='1400 MISSION ST3507039']

#there are 2 101 EXECUTIVE PARK BL's. I believe that they are separate projects so leaving them so here.

#duplicates. Updating comp date to most recent info. Updating construction date to earliest info.
new_df.loc[(new_df['address']=='1 ECKER ST') & (new_df['apn']=='3708022'), 'con_date'] = '2009-05-12'
new_df=new_df[new_df['address_apn'] !='1 ECKER ST3708056']


# Explore Duplicate Data

In [117]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address_apn'].unique()


array(['1006 16TH ST3833002',
       '800 BROTHERHOOD WAY (UNDER CONSTRUCTION)7331005',
       'HUDSON AVE AND WHITNEY YOUNG CIR4711118',
       '1201-1225 TENNESSEE STREET4172022', '800 BROTHERHOOD WAY7331005',
       '145 AMES ST3632038', '1000 16TH ST3833001',
       '236 - 238 SHIPLEY ST / 77 FALMOUTH ST3753314',
       '55 LAGUNA ST (BLDG 1)0870003',
       '1006 / 1050 16TH ST & 1380 07TH ST3833002',
       '1068 GUERRERO ST3632039', '101 EXECUTIVE PARK BL4991279',
       '800 BROTHERHOOD WAY7331003', '1201 - 1225 TENNESSEE STREET4172022',
       '5 SHIPLEY ST / 935 FOLSOM ST / 77 FALMOUTH ST3753314',
       '1400 07TH ST3833001', '83 PANORAMA DR2821010',
       '100 BUCHANAN ST0870003', '201 FOLSOM ST3746003',
       '201 FOLSOM ST3746001', '1340-1390 MISSION ST3508053',
       'HUNTERS POINT SHIPYARD PHASE I4591C001',
       '1634-1690 PINE ST0647007', '101 EXECUTIVE PARK BL4991617',
       '83 PANORAMA DR2821023', '5800 03RD ST (BUILDING 4)5431A042',
       '1634 - 1690 PINE S

In [131]:
new_df[new_df['address_apn'] == '2101 & 2155 WEBSTER STREET']['dbi_permit']

Series([], Name: dbi_permit, dtype: object)

In [119]:
#new_df[pd.isnull(new_df['dbi_permit'])]

In [121]:
new_df[new_df['dbi_permit']==201112070234.00]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,address_apn
16492,2012-09-07,1006 16TH ST,3833002,2014-09-30,CONSTRUCTION,10/01/2014,2014-09-30,201112070234.00,2011-12-07,2011-12-07,2011-12-07 00:00:00,2014-09-30,CONSTRUCTION,"('2011-12-07', '2011-12-07', '2011-12-07', '20...",1029.00,"('BP FILED', 'BP FILED', 'BP FILED', 'BP ISSUE...",3,2014,Under Construction,393.00,393.00,-122.40,37.77,1006 16TH ST3833002
22549,,1006 / 1050 16TH ST & 1380 07TH ST,3833002,2016-03-30,CONSTRUCTION,04/01/2016,2016-03-30,201112070234.00,2003-05-09,2016-03-30,2003-05-09,2016-03-30,CONSTRUCTION,"('2016-03-30',)",4711.00,"('CONSTRUCTION',)",1,2016,Under Construction,393.00,393.00,-122.40,37.77,1006 / 1050 16TH ST & 1380 07TH ST3833002


In [41]:
new_df[new_df['address_apn']=='HUNTERS VIEW4624031']['firstfiled']

29283    
Name: firstfiled, dtype: object

In [ ]:
#explore duplicated
new_df[new_df['address']=='2155 WEBSTER ST']

In [ ]:
new_df[new_df['dbi_permit']=='200705010136']

In [ ]:
new_df[new_df['project_duration_days']<0]

# Manual Data Cleaning
Based off of identified duplicates in the previous section

In [ ]:
#this has a duplicate, same address, different APN
new_df=new_df[(new_df['address']!='3575 GEARY BL') | (new_df['apn']=='1084010')]

In [ ]:
#same project, happening at different periods
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007'), 'comp_date']='07/01/2016'
new_df=new_df[(new_df['address']!='1634 - 1690 PINE ST') | (new_df['apn']=='0647007')]

In [ ]:
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007')]

In [ ]:
#Be sure to redo project duration after all manual cleaning

In [ ]:
#do median time based on neighborhood

In [ ]:
#breakdown time to completion by stage for those projects that this can be done for